In [294]:
v5dir = "/home/yolov5/yolov5/"
exp = "exp34"
dataset_dir = "/data_dataset_v5/" #relative to yolov5/datasets/ folder
dataset_yaml = "/dataset_v5.yaml"
tst_or_val = "test" #run on the test or validation set, might be train also
last_or_best = "best" #trained model weights 

In [295]:
#modifying yaml file to test or validation in 'val' field
import yaml
import io

with open(v5dir+"../datasets/"+dataset_dir+"/"+dataset_yaml, 'r') as stream:
    data = yaml.safe_load(stream)
    data['val'] = "images/"+tst_or_val
print(data)

classes = ",".join(data["names"])
print(classes)

with io.open(v5dir+"../datasets/"+dataset_dir+"/"+dataset_yaml, 'w', encoding='utf8') as outfile:
    yaml.dump(data, outfile, default_flow_style=False, allow_unicode=True)

{'path': '/home/yolov5/datasets/data_dataset_v5/', 'train': 'images/train', 'val': 'images/test', 'test': 'images/test', 'nc': 2, 'names': ['adult', 'kid']}
adult,kid


In [296]:
!rm -rf input/detection-results/*
!rm -rf input/ground-truth/*
!rm -rf {v5dir}runs/val/*

In [297]:
!python3 {v5dir}val.py --data {v5dir}../datasets/{dataset_dir}/{dataset_yaml} --weights {v5dir}runs/train/{exp}/weights/{last_or_best}.pt --batch 8 --save-txt --save-conf

val: data=/home/yolov5/yolov5/../datasets//data_dataset_v5///dataset_v5.yaml, weights=['/home/yolov5/yolov5/runs/train/exp34/weights/best.pt'], batch_size=8, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=../yolov5/yolov5/runs/val, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v6.0-8-g1172ac1 torch 1.10.0+cu102 CUDA:0 (GeForce RTX 2080, 7981.0MB)

Fusing layers... 
/usr/local/lib/python3.8/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 290 layers, 20856975 parameters, 0 gradients, 48.0 GFLOPs
val: Scanning '/home/yolov5/datasets/data_dataset_v5/labels/test' images and lab
val: New cache created: /ho

In [298]:
#converting detection results
!python3 scripts/extra/conv_gt_dr_yol5_to_repo_format.py --img-dir {v5dir}../datasets/{dataset_dir}/images/{tst_or_val}/ --gt-dr-dir {v5dir}/runs/val/exp/labels/ --save-dir input/detection-results/ --dt

In [299]:
#converting dataset validation ground truths
!python3 scripts/extra/conv_gt_dr_yol5_to_repo_format.py --img-dir {v5dir}../datasets/{dataset_dir}/images/{tst_or_val}/ --gt-dr-dir {v5dir}../datasets/{dataset_dir}/labels/{tst_or_val}/ --save-dir input/ground-truth/

In [300]:
!python3 scripts/extra/intersect-gt-and-dr.py

total ground-truth files: 239
total detection-results files: 239

No backup required for /home/mAP/input/ground-truth
No backup required for /home/mAP/input/detection-results
total intersected files: 239
Intersection completed!


In [301]:
#this are results that will go into comparation table
!python3 main.py -na -q --no-plot --classes {classes} --train-gt-folder {v5dir}../datasets/{dataset_dir}/labels/train/

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95
                 all        239        469      0.846      0.745      0.817      0.573
----------------------------------------------------------------------------------------------------
               adult        239        183      0.818      0.661      0.729      0.461
                 kid        239        286      0.875      0.829      0.905      0.684


labels per class in train set:
adult,kid
{'0': 3223, '1': 5185}



----- for easy insertion into a sheet just copy this by row
all	239	469	0.846	0.745	0.817	0.573
adult	239	183	0.818	0.661	0.729	0.461
kid	239	286	0.875	0.829	0.905	0.684
